## **Fine-Tuning a Generative AI Model for Personality detection**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requir

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

## Loading Dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'/content/drive/MyDrive/Personality_detection_Finetune/Mypersonality1.csv'},encoding='latin1',split='train')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1a83b80bdf7fa4dc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [ ]:
dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/Personality_detection_Finetune/train.csv',
                                              'test': '/content/drive/MyDrive/Personality_detection_Finetune/test.csv','validation':'/content/drive/MyDrive/Personality_detection_Finetune/validation.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-172b242ec4f9cb2a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['STATUS', 'extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness', 'Personality'],
        num_rows: 5950
    })
    test: Dataset({
        features: ['STATUS', 'extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness', 'Personality'],
        num_rows: 1984
    })
    validation: Dataset({
        features: ['STATUS', 'extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness', 'Personality'],
        num_rows: 1983
    })
})

## Importing flan-t5 model

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


## Test the Model with Zero Shot Inferencing

In [ ]:
index = 200

dialogue = dataset['test'][index]['STATUS']
summary = dataset['test'][index]['Personality']

prompt = f"""
Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :

GO STATE BEAT THE BOBCATS!!!!!

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
agreeableness

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Conscientiousness


## Full Fine-Tuning

Preprocess the Dataset

In [ ]:
def tokenize_function(example):
    start_prompt = 'Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :\n\n'
    end_prompt = '\n\Personality: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["STATUS"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["Personality"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['STATUS', 'extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness', 'Personality'])

Map:   0%|          | 0/5950 [00:00<?, ? examples/s]

Map:   0%|          | 0/1984 [00:00<?, ? examples/s]

Map:   0%|          | 0/1983 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/5950 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1984 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1983 [00:00<?, ? examples/s]

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")
print(tokenized_datasets)

Shapes of the datasets:
Training: (60, 2)
Validation: (20, 2)
Test: (20, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 60
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 20
    })
})


In [ ]:
output_dir='/content/drive/MyDrive/Personality_detection_Finetune/LLM_finetune'
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,40.750000


TrainOutput(global_step=1, training_loss=40.75, metrics={'train_runtime': 6.3272, 'train_samples_per_second': 1.264, 'train_steps_per_second': 0.158, 'total_flos': 5478058819584.0, 'train_loss': 40.75, 'epoch': 0.12})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Personality_detection_Finetune/LLM_finetune")

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Personality_detection_Finetune/LLM_finetune", torch_dtype=torch.bfloat16)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model.to(device)
instruct_model.to(device)
input_ids = input_ids.to(device)
index = 530

dialogue = dataset['test'][index]['STATUS']
human_baseline_summary = dataset['test'][index]['Personality']

prompt = f"""
Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :

{dialogue}

Presonality Trait:
"""



original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
agreeableness
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Agreeableness
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Agreeableness


## Evaluating The model

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
dialogues = dataset['test'][0:10]['STATUS']
human_baseline_summaries = dataset['test'][0:10]['Personality']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
original_model.to(device)
instruct_model.to(device)
input_ids = input_ids.to(device)
original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :

{dialogue}

Summary: """


    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,openness,Predicting the following factors:,Agreeableness
1,openness,:,Agreeableness
2,openness,Unconscientiousness,Agreeableness
3,openness,RIP,Agreeableness
4,agreeableness,a).,Agreeableness
5,openness,Conscientiousness,Agreeableness
6,openness,RIP,Agreeableness
7,openness,Neuroticism,Agreeableness
8,neuroticism,Neuroticism,Agreeableness
9,conscientiousness,Agreeableness,Agreeableness


In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.1, 'rouge2': 0.0, 'rougeL': 0.1, 'rougeLsum': 0.1}
INSTRUCT MODEL:
{'rouge1': 0.1, 'rouge2': 0.0, 'rougeL': 0.1, 'rougeLsum': 0.1}


## Parameter Efficient Fine-Tuning (PEFT)

### LoRA  for Fine-Tuning

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


## Training

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [ ]:
peft_trainer.train()

peft_model_path="/content/drive/MyDrive/Personality_detection_Finetune/peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,40.250000


('/content/drive/MyDrive/Personality_detection_Finetune/peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 '/content/drive/MyDrive/Personality_detection_Finetune/peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 '/content/drive/MyDrive/Personality_detection_Finetune/peft-dialogue-summary-checkpoint-local/tokenizer.json')

## PEFT model Evaluation

In [ ]:
from peft import PeftModel, PeftConfig
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [ ]:
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       '/content/drive/MyDrive/Personality_detection_Finetune/peft-dialogue-summary-checkpoint-local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


In [ ]:
index = 200
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
original_model.to(device)
instruct_model.to(device)
peft_model.to(device)
input_ids = input_ids.to(device)
dialogue = dataset['test'][index]['STATUS']
baseline_human_summary = dataset['test'][index]['Personality']

prompt = f"""
Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :


{dialogue}

Summary: """


original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
agreeableness
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Neuroticism
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Agreeableness
---------------------------------------------------------------------------------------------------
PEFT MODEL: Agreeableness


In [ ]:
dialogues = dataset['test'][0:10]['STATUS']
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

human_baseline_summaries = dataset['test'][0:10]['Personality']
device = torch.device("cuda:0")
original_model.to(device)
instruct_model.to(device)
peft_model.to(device)
input_ids = input_ids.to(device)

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Predict whether the above sentence is Openness or Conscientiousness or Extraversion or Agreeableness or Neuroticism :

{dialogue}

Summary: """


    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,openness,: GO STATE BEACH!!!!!,Conscientiousness,Conscientiousness
1,openness,Conscientiousness,Conscientiousness,Conscientiousness
2,openness,Agreeableness,Conscientiousness,Conscientiousness
3,openness,Agreeableness,Conscientiousness,Conscientiousness
4,agreeableness,Neuroticism,Conscientiousness,Conscientiousness
5,openness,Agreeable,Conscientiousness,Conscientiousness
6,openness,GO STATE BATTLES THE BOBCATS,Conscientiousness,Conscientiousness
7,openness,GO STATE BACK THE BOBCATS!!!!!,Conscientiousness,Conscientiousness
8,neuroticism,: GO STATE BACK THE BOBCATS!!!!!,Conscientiousness,Conscientiousness
9,conscientiousness,Agreeableness,Conscientiousness,Conscientiousness


In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
INSTRUCT MODEL:
{'rouge1': 0.1, 'rouge2': 0.0, 'rougeL': 0.1, 'rougeLsum': 0.1}
PEFT MODEL:
{'rouge1': 0.1, 'rouge2': 0.0, 'rougeL': 0.1, 'rougeLsum': 0.1}
